In [1]:
P = Primes()
D.<x> = PolynomialRing(QQ)
R.<x> = PolynomialRing(QQ)
C.<x> = PolynomialRing(CC)
S = PowerSeriesRing(QQ,'x', default_prec=300)
#S.<x,y>=PowerSeriesRing(QQ,default_prec=300)
# Using "R = PolynomialRing(QQ,'x')" produces an error.
# Yet using S as defined here is fine (see definition of Fraleigh4 below)
# But S = PowerSeriesRing(QQ,'x','y') also produces an error.
def pochhammer(x,n):
    return product ([x+k for k in [0..n-1]])
def cRaleigh2(alpha,beta,nu):
    pchhmrA = pochhammer(alpha,nu)
    pchhmrB = pochhammer(beta,nu)
    fnu=factorial(nu)
    answer = D(pchhmrA*pchhmrB/fnu^2)
    return answer
def term(alpha,beta,p):
    return D(1/(alpha + p) + 1/(beta + p) - 2/(1 + p))
def eRaleigh(alpha, beta, nu):
    return D(sum([term(alpha, beta, p) for p in [0..nu - 1]]))
def FstarRaleigh2(alpha,beta,u,terms):
    fsr = sum([cRaleigh2(alpha, beta, nu)*eRaleigh(alpha,beta,nu)*u^nu for nu in [1..terms]])
    return D(fsr)
def Fraleigh2(alpha,beta,u,terms): 
    return D(sum([cRaleigh2(alpha,beta,nu)*u^nu for nu in [0..terms]]))
def FstarRaleigh3(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fsr2 = FstarRaleigh2(alpha,beta,x,n)
    return D(fsr2)
def Fraleigh4(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fr2 = Fraleigh2(alpha,beta,x,n)
    return D(fr2)
def exNo3c(n,m,x):
    a1 = S(x)*exp(S(FstarRaleigh3(n,m,x)/Fraleigh4(n,m,x))).O(2*n)
    # S(x) instead of x in df of a1 to avoid an error message
    # after use of the stripQuotationMarks command below.
    a2 = S(a1).O(n+1)
    return a2 # applying S to argument of exp is key.
def J(n,m,x):
    exn = exNo3c(n+2,m,x)
    jnm = 1/exn.reverse()
    return jnm.O(n+1) # Jtest is faster than J, and they agree.
def normalJ(n,m,x):
    f = J(n,m,x)
    return f.truncate(n+1)
def Jcoefficient(n,m,x):
    f = J(n+2,m,x)
    f1 = f.truncate(n+1)
    f1=f1.list()
    f1=Set(f1)
    g1 = f.truncate(n)
    g1 = g1.list()
    g1 = set(g1)
    h = f1.difference(g1)
    h = h.list()
    return h[0]
def xjNewStep(n,m,x): 
    f=J(n,m,x)
    fl=f.list()
    return (sum(c*x**i*(m**3*2**6)**i for i,c in enumerate(fl)))
def xjNew(n,m,x):
    return xjNewStep(n-1,m,x)
def xjNewCoefficient(n,m,x):
    if n>0:return (xjNew(n,m,x)-xjNew(n-1,m,x))/x^n
    if n==0:return 1
def xjNewTest(n,m,x):
    return D(xjNewStep(n-1,m,x))
def polynomialCoefficient(n,poly):
    return poly.list()[n]
def factorpairToFactor(factorpair):
    (monomial,exponent)=factorpair
    return monomial^exponent
def numericalfactor(polynomial):
    fp=polynomial.factor()
    lst=[factorpairToFactor(factorpair) for factorpair in fp]
    return polynomial/product(lst)
def lastfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.last())
def reduced(polynomial):
    return numericalfactor(polynomial)*lastfactor(polynomial)
def stripQuotationMarks(string):
    var('x')
    return sage_eval(string,locals={'x':x})

def prime_n(a):return P.unrank(a-1)

def primeFactors(n):
    answer=[]
    lf=list(n.factor())
    length=len(lf)
    for k in [0..length-1]:
        pair=lf[k]
        (base,exponent)=pair
        answer=answer+[[base]]
    return(flatten(answer))

def primeFactorsInList(lst):
    ld=len(lst)
    primelist=[]
    for k in [0..ld-1]:
        a=ZZ(lst[k])
        primelist=primelist+[primeFactors(a)]
    primelist=flatten(primelist)
    primelist=list(set(primelist))
    return primelist

def den(a):
    if a in QQ:
        if a in ZZ:return 1
        return a.denominator()
def listdens(lst):
    return list(den(a) for a in lst)
def polynomialdenominator(poly):return lcm(listdens(poly))
def polynomialnumerator(poly):
    return expand(poly*polynomialdenominator(poly))
def num(a):return a.numerator()
def prime(n):
    P=Primes()
    return P.unrank(n-1)
def inclusiveListPrimesTo(n):
    if is_prime(n):
        return list(primes(n+1))
    else:
        return list(primes(n))
def chunk(k,list):
    chnk=[]
    for n in [k..len(list)-1]:
        chnk=chnk+[list[n]]
    return chnk
def drop(m,lst): return list(Set(lst).difference({m}))
def sort(list):
    answr=[]
    chnk=list
    for n in [0..len(list)-1]:
        m = min(chnk)
        answr=answr+[m]
        chnk=drop(m,chnk)
    return answr
def ord(p,n):return QQ.valuation(p)(n)
def base_conversion(number, base_from, base_to):  # by Vishnu Namboothiri
# https://ask.sagemath.org/question/9657/how-to-output-decimal-numbers-in-different-radix/
    if(base_from==10):
        return stripQuotationMarks((number.str(base=base_to)))
    else:
        decimal = Integer(str(number), base=base_from)
        return (stripQuotationMarks(decimal.str(base=base_to)))
def lastFactorWithIntegralFactors(polynomial):
    return polynomialnumerator(lastfactor(polynomial))
def numericalFactorWithIntegralFactors(polynomial):
    nf=numericalfactor(polynomial);
    pd=polynomialdenominator(lastfactor(polynomial))
    return nf/pd


def primeDivisors(n):
    pd=[]
    n=n
    fn=factor(n)
    ln=len(fn)
    for k in [0..ln-1]:
        part = fn[k]
        (prime,exponent) = part
        pd = pd + [prime]
    return pd 
def fOverField(f,prime,power):
    answer=[]
    T = GF(prime^power)
    G.<t>= GF(prime^power)
    U.<x> = T[]
    ppoly = U(f)
    expr=str(ppoly)
    var('x')
    fp=ppoly.factor()
    return ppoly

###################### following two routines disagree ####################
def factorOverField(f,prime,power):
    answer=[]
    T = GF(prime^power,'t')
    G.<t>= GF(prime^power)
    U.<x> = T[]
    ppoly = U(f)
    expr=str(ppoly)
    var('x')
    fp=ppoly.factor()
    return fp

def factorOverField2(poly,prime,power,j):
    sd = leastSplittingDegree(poly,prime,bound)
    T=GF(prime^j)
    G.<t>= GF(prime^sd)
    U.<x> = T[]
    ppoly=U(poly)
    fpp=ppoly.factor()
    return fpp

############################

#######################################################
# splitting tests below fail at  e.g. f = x + 1 
############################################################

def splitsTF(f,prime,power):
    T = GF(prime^power)
    U.<x> = T[]
    g=U(f)
    gf=g.factor()
    lnf = len(gf)-1
    for nn in [0..lnf]:
        part = gf[nn]
        (monomial,exponent) = part
        mdx = monomial.degree()
        if mdx > 1:return(False)
    return(True)

def leastSplittingDegree(f,prime,bound):
    for power in [1..bound]:
        tf=splitsTF(f,prime,power)
        if tf == True:return power
    return -1

#########################################################

def firstNprimes(n):
    P=Primes()
    first=[]
    for k in [0..n-1]:
        first=first+[P.unrank(k)]
    return first

def slopeExistsTF(pairlist):
    lenth=len(pairlist)
    if lenth<3:return False
    slopes=[]
    for k in [1..lenth-1]:
        pair1=pairlist[k-1]
        abscissa1=pair1[0]
        ordinate1=pair1[1]
        pair2=pairlist[k]
        abscissa2=pair2[0]
        ordinate2=pair2[1]
        dx=abscissa2-abscissa1
        if dx == 0:return False
        dy=ordinate2-ordinate1
        slope=dy/dx
        slopes=slopes+[slope]
    slopes=Set(slopes)
    if len(slopes) == 1:
        return [True,slopes[0]]
    if len(slopes) > 1:
        return False 
    
def zeroOfLinearMonomial(monomial,x):return x-monomial

def totalDegreeOverField(f,prime,power):
    T = GF(prime^power,'t')
    U.<x> = T[]
    g=U(f)
    gf=g.factor()
    lnf = len(gf)-1
    td=0
    for nn in [0..lnf]:
        part = gf[nn]
        (monomial,exponent) = part
        td=td+exponent
    return td

def dropzeros(list):
    data=[]
    el=len(list)
    for j  in [0..el-1]:
        lj=list[j]
        if lj != 0:data=data+[lj]
    return data

In [ ]:
## printed output has been snipped

import pickle
wfile=open('/Users/barrybrent/run1jun21no1.txt','a')
data=[]
for m in [3..500]:
    print m
    data=data+[[m,normalJ(200,m)]]
s=pickle.dumps(str(data))
wfile.write(s)
wfile.close()
#########################
import pickle
rfile = open('/Users/barrybrent/run1jun21no1.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)

polydata=[]

wfile = open('/Users/barrybrent/run14jun21no14.txt','a') # xJ polynomials
for n in [0..200]:
    data=[]
    for k in [0..2*n+1]:
        print (n,k)
        m = s[k][0]
        poly=x*s[k][1] 
        cf=polynomialCoefficient(n,poly)
        data=data+[[m,cf*m^(2*n)]]
    rl=R.lagrange_polynomial(data)
    polydata=polydata+[[n-1,rl]] 

    
t = pickle.dumps(str(polydata))
wfile.write(t)
wfile.close() # print-output snipped

In [3]:
import pickle
rfile = open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
print len(s)
print s[1]

201
[0, 3/8*x^2 + 1/2]


In [4]:
import pickle
rfile = open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
bound=12
for k in [1..6]:
    index=s[k][0]
    poly = s[k][1]
    print poly

3/8*x^2 + 1/2
69/1024*x^4 - 1/128*x^2 - 3/64
1/128*x^6 - 29/864*x^4 + 1/216*x^2 + 1/54
5601/8388608*x^8 - 3821/524288*x^6 + 5195/262144*x^4 - 101/32768*x^2 - 303/32768
3/65536*x^10 - 16367/18432000*x^8 + 7961/1382400*x^6 - 21809/1728000*x^4 + 373/172800*x^2 + 373/72000
23003/8589934592*x^12 - 2254159/28991029248*x^10 + 338577733/391378894848*x^8 - 106154827/24461180928*x^6 + 68420351/8153726976*x^4 - 4754693/3057647616*x^2 - 4754693/1528823808


In [5]:
import pickle
rfile = open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
bound=12
for k in [1..6]:
    index=s[k][0]
    poly = s[k][1]
    print polynomialnumerator(D(poly))

3*x^2 + 4
69*x^4 - 8*x^2 - 48
27*x^6 - 116*x^4 + 16*x^2 + 64
5601*x^8 - 61136*x^6 + 166240*x^4 - 25856*x^2 - 77568
10125*x^10 - 196404*x^8 + 1273760*x^6 - 2791552*x^4 + 477440*x^2 + 1145856
16769187*x^12 - 486898344*x^10 + 5417243728*x^8 - 27175635712*x^6 + 52546829568*x^4 - 9737611264*x^2 - 19475222528


In [40]:
def numericalTermOverField(f,prime,power):
    T = GF(prime^power,'t')
    U.<x> = T[]
    g=U(f)
    gf=g.factor()
    print gf
    lnf = len(gf)-1
    xproduct=1
    for nn in [0..lnf]:
        part = gf[nn]
        (monomial,exponent) = part
        xproduct=xproduct*monomial^exponent
    return g/xproduct


def numericalTermOverFieldNoPrint(f,prime,power):
    T = GF(prime^power,'t')
    U.<x> = T[]
    g=U(f)
    gf=g.factor()
    lnf = len(gf)-1
    xproduct=1
    for nn in [0..lnf]:
        part = gf[nn]
        (monomial,exponent) = part
        xproduct=xproduct*monomial^exponent
    return g/xproduct

In [5]:
import pickle
rfile = open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
bound=120
prime=5
for k in [1..6]:
    index=s[k][0]
    poly = s[k][1]
    poly=polynomialnumerator(D(poly))
    sd=leastSplittingDegree(poly,prime,bound)
print numericalTermOverField(poly,prime,sd)

(2) * (x + 2)^2 * (x + 3)^2 * (x + 1)^4 * (x + 4)^4
2


In [7]:
import pickle
rfile = open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
bound=120
prime=5
residue=0
data=[]
for k in [1..len(s)-1]:
    index=s[k][0]
    if mod(index,prime)==residue:
        poly = s[k][1]
        poly=polynomialnumerator(D(poly))
        sd=leastSplittingDegree(poly,prime,bound)
        nt=numericalTermOverField(poly,prime,sd)
        print (index,sd,nt)
        data=data+[nt]
print data

(3) * (x + t + 2) * (x + 4*t + 3)
(0, 2, 3)
(2) * (x + 2)^2 * (x + 3)^2 * (x + 1)^4 * (x + 4)^4
(5, 1, 2)
(x + 2)^4 * (x + 3)^4 * (x + 1)^6 * (x + 4)^6
(10, 1, 1)
(4) * (x + 2)^6 * (x + 3)^6 * (x + 1)^8 * (x + 4)^8
(15, 1, 4)
(x + 2)^8 * (x + 3)^8 * (x + 1)^10 * (x + 4)^10
(20, 1, 1)
(3) * (x + 2)^10 * (x + 3)^10 * (x + 1)^12 * (x + 4)^12
(25, 1, 3)
(x + 2)^12 * (x + 3)^12 * (x + 1)^14 * (x + 4)^14
(30, 1, 1)
(3) * (x + 2)^14 * (x + 3)^14 * (x + 1)^16 * (x + 4)^16
(35, 1, 3)
(4) * (x + 2)^16 * (x + 3)^16 * (x + 1)^18 * (x + 4)^18
(40, 1, 4)
(3) * (x + 2)^18 * (x + 3)^18 * (x + 1)^20 * (x + 4)^20
(45, 1, 3)
(3) * (x + 2)^20 * (x + 3)^20 * (x + 1)^22 * (x + 4)^22
(50, 1, 3)
(x + 2)^22 * (x + 3)^22 * (x + 1)^24 * (x + 4)^24
(55, 1, 1)
(3) * (x + 2)^24 * (x + 3)^24 * (x + 1)^26 * (x + 4)^26
(60, 1, 3)
(4) * (x + 2)^26 * (x + 3)^26 * (x + 1)^28 * (x + 4)^28
(65, 1, 4)
(3) * (x + 2)^28 * (x + 3)^28 * (x + 1)^30 * (x + 4)^30
(70, 1, 3)
(3) * (x + 2)^30 * (x + 3)^30 * (x + 1)^32 * (x + 4)^32
(

In [8]:
import pickle
rfile = open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
bound=120
prime=5

data=[]
for k in [1..len(s)-1]:
    index=s[k][0]
   
    poly = s[k][1]
    poly=polynomialnumerator(D(poly))
    sd=leastSplittingDegree(poly,prime,bound)
    nt=numericalTermOverField(poly,prime,sd)
    print (index,sd,nt)
    data=data+[nt]
print data

(3) * (x + t + 2) * (x + 4*t + 3)
(0, 2, 3)
(4) * (x + 2*t^3 + 4*t + 1) * (x + 2*t^3 + t^2 + t + 3) * (x + 3*t^3 + t + 4) * (x + 3*t^3 + 4*t^2 + 4*t + 2)
(1, 4, 4)
(2) * (x + 2) * (x + 3) * (x + t^2 + 2*t + 2) * (x + 4*t^2 + 3*t + 3) * (x + t^3 + 4*t^2 + 1) * (x + 4*t^3 + t^2 + 4)
(2, 4, 2)
(x + 2) * (x + 3) * (x + 2*t^3 + 3*t^2 + 2*t) * (x + 3*t^3 + 2*t^2 + 3*t) * (x + 2*t^5 + 3*t^3 + 3*t + 1) * (x + 2*t^5 + 2*t^4 + t^2 + t + 1) * (x + 3*t^5 + 2*t^3 + 2*t + 4) * (x + 3*t^5 + 3*t^4 + 4*t^2 + 4*t + 4)
(3, 6, 1)
(x + 1)^2 * (x + 2)^2 * (x + 3)^2 * (x + 4)^2
(4, 1, 1)
(2) * (x + 2)^2 * (x + 3)^2 * (x + 1)^4 * (x + 4)^4
(5, 1, 2)
(3) * (x + 2*t^3 + 4*t + 1) * (x + 2*t^3 + t^2 + t + 3) * (x + 3*t^3 + t + 4) * (x + 3*t^3 + 4*t^2 + 4*t + 2) * (x + 1)^2 * (x + 2)^2 * (x + 3)^2 * (x + 4)^2
(6, 4, 3)
(2) * (x + t^2 + 2*t + 2) * (x + 4*t^2 + 3*t + 3) * (x + t^3 + 4*t^2 + 1) * (x + 4*t^3 + t^2 + 4) * (x + 1)^2 * (x + 4)^2 * (x + 2)^3 * (x + 3)^3
(7, 4, 2)
(2) * (x + 2*t^3 + 3*t^2 + 2*t) * (x + 3*t

(3) * (x + 2*t^3 + 4*t + 1) * (x + 2*t^3 + t^2 + t + 3) * (x + 3*t^3 + t + 4) * (x + 3*t^3 + 4*t^2 + 4*t + 2) * (x + 1)^24 * (x + 2)^24 * (x + 3)^24 * (x + 4)^24
(61, 4, 3)
(2) * (x + t^2 + 2*t + 2) * (x + 4*t^2 + 3*t + 3) * (x + t^3 + 4*t^2 + 1) * (x + 4*t^3 + t^2 + 4) * (x + 1)^24 * (x + 4)^24 * (x + 2)^25 * (x + 3)^25
(62, 4, 2)
(4) * (x + 2*t^3 + 3*t^2 + 2*t) * (x + 3*t^3 + 2*t^2 + 3*t) * (x + 2*t^5 + 3*t^3 + 3*t + 1) * (x + 2*t^5 + 2*t^4 + t^2 + t + 1) * (x + 3*t^5 + 2*t^3 + 2*t + 4) * (x + 3*t^5 + 3*t^4 + 4*t^2 + 4*t + 4) * (x + 1)^24 * (x + 4)^24 * (x + 2)^25 * (x + 3)^25
(63, 6, 4)
(3) * (x + 1)^26 * (x + 2)^26 * (x + 3)^26 * (x + 4)^26
(64, 1, 3)
(4) * (x + 2)^26 * (x + 3)^26 * (x + 1)^28 * (x + 4)^28
(65, 1, 4)
(2) * (x + 2*t^3 + 4*t + 1) * (x + 2*t^3 + t^2 + t + 3) * (x + 3*t^3 + t + 4) * (x + 3*t^3 + 4*t^2 + 4*t + 2) * (x + 1)^26 * (x + 2)^26 * (x + 3)^26 * (x + 4)^26
(66, 4, 2)
(4) * (x + t^2 + 2*t + 2) * (x + 4*t^2 + 3*t + 3) * (x + t^3 + 4*t^2 + 1) * (x + 4*t^3 + t^2 + 4

(4) * (x + 1)^48 * (x + 2)^48 * (x + 3)^48 * (x + 4)^48
(119, 1, 4)
(4) * (x + 2)^48 * (x + 3)^48 * (x + 1)^50 * (x + 4)^50
(120, 1, 4)
(x + 2*t^3 + 4*t + 1) * (x + 2*t^3 + t^2 + t + 3) * (x + 3*t^3 + t + 4) * (x + 3*t^3 + 4*t^2 + 4*t + 2) * (x + 1)^48 * (x + 2)^48 * (x + 3)^48 * (x + 4)^48
(121, 4, 1)
(4) * (x + t^2 + 2*t + 2) * (x + 4*t^2 + 3*t + 3) * (x + t^3 + 4*t^2 + 1) * (x + 4*t^3 + t^2 + 4) * (x + 1)^48 * (x + 4)^48 * (x + 2)^49 * (x + 3)^49
(122, 4, 4)
(4) * (x + 2*t^3 + 3*t^2 + 2*t) * (x + 3*t^3 + 2*t^2 + 3*t) * (x + 2*t^5 + 3*t^3 + 3*t + 1) * (x + 2*t^5 + 2*t^4 + t^2 + t + 1) * (x + 3*t^5 + 2*t^3 + 2*t + 4) * (x + 3*t^5 + 3*t^4 + 4*t^2 + 4*t + 4) * (x + 1)^48 * (x + 4)^48 * (x + 2)^49 * (x + 3)^49
(123, 6, 4)
(2) * (x + 1)^50 * (x + 2)^50 * (x + 3)^50 * (x + 4)^50
(124, 1, 2)
(4) * (x + 2)^50 * (x + 3)^50 * (x + 1)^52 * (x + 4)^52
(125, 1, 4)
(x + 2*t^3 + 4*t + 1) * (x + 2*t^3 + t^2 + t + 3) * (x + 3*t^3 + t + 4) * (x + 3*t^3 + 4*t^2 + 4*t + 2) * (x + 1)^50 * (x + 2)^50 * (x

(4) * (x + t^2 + 2*t + 2) * (x + 4*t^2 + 3*t + 3) * (x + t^3 + 4*t^2 + 1) * (x + 4*t^3 + t^2 + 4) * (x + 1)^70 * (x + 4)^70 * (x + 2)^71 * (x + 3)^71
(177, 4, 4)
(4) * (x + 2*t^3 + 3*t^2 + 2*t) * (x + 3*t^3 + 2*t^2 + 3*t) * (x + 2*t^5 + 3*t^3 + 3*t + 1) * (x + 2*t^5 + 2*t^4 + t^2 + t + 1) * (x + 3*t^5 + 2*t^3 + 2*t + 4) * (x + 3*t^5 + 3*t^4 + 4*t^2 + 4*t + 4) * (x + 1)^70 * (x + 4)^70 * (x + 2)^71 * (x + 3)^71
(178, 6, 4)
(x + 1)^72 * (x + 2)^72 * (x + 3)^72 * (x + 4)^72
(179, 1, 1)
(4) * (x + 2)^72 * (x + 3)^72 * (x + 1)^74 * (x + 4)^74
(180, 1, 4)
(3) * (x + 2*t^3 + 4*t + 1) * (x + 2*t^3 + t^2 + t + 3) * (x + 3*t^3 + t + 4) * (x + 3*t^3 + 4*t^2 + 4*t + 2) * (x + 1)^72 * (x + 2)^72 * (x + 3)^72 * (x + 4)^72
(181, 4, 3)
(3) * (x + t^2 + 2*t + 2) * (x + 4*t^2 + 3*t + 3) * (x + t^3 + 4*t^2 + 1) * (x + 4*t^3 + t^2 + 4) * (x + 1)^72 * (x + 4)^72 * (x + 2)^73 * (x + 3)^73
(182, 4, 3)
(x + 2*t^3 + 3*t^2 + 2*t) * (x + 3*t^3 + 2*t^2 + 3*t) * (x + 2*t^5 + 3*t^3 + 3*t + 1) * (x + 2*t^5 + 2*t^4 +

In [9]:
import pickle
rfile = open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
bound=120
prime=5
sd=12
data=[]
for k in [1..len(s)-1]:
    index=s[k][0]
   
    poly = s[k][1]
    poly=polynomialnumerator(D(poly))
   
    nt=numericalTermOverField(poly,prime,sd)
    print (index,sd,nt)
    data=data+[nt]
print data

(3) * (x + t^11 + t^10 + 3*t^9 + 2*t^7 + t^6 + t^5 + 2*t^4 + 2*t^3 + t^2 + t) * (x + 4*t^11 + 4*t^10 + 2*t^9 + 3*t^7 + 4*t^6 + 4*t^5 + 3*t^4 + 3*t^3 + 4*t^2 + 4*t)
(0, 12, 3)
(4) * (x + t^11 + 4*t^10 + t^9 + 3*t^8 + 4*t^7 + 2*t^6 + t^5 + 4*t^4 + t^2 + 2*t) * (x + 4*t^11 + t^10 + 4*t^9 + 2*t^8 + t^7 + 3*t^6 + 4*t^5 + t^4 + 4*t^2 + 3*t) * (x + 2*t^11 + 3*t^10 + 2*t^9 + 4*t^8 + 2*t^6 + 2*t^5 + t^4 + 4*t^3 + t^2 + 4*t + 2) * (x + 3*t^11 + 2*t^10 + 3*t^9 + t^8 + 3*t^6 + 3*t^5 + 4*t^4 + t^3 + 4*t^2 + t + 3)
(1, 12, 4)
(2) * (x + 2) * (x + 3) * (x + 3*t^11 + 2*t^10 + 3*t^9 + 2*t^8 + 4*t^7 + 4*t^6 + 3*t^5 + 4*t^3 + 2*t^2 + t + 2) * (x + t^11 + 4*t^10 + t^9 + t^8 + t^7 + t^5 + 2*t^4 + 4*t^3 + 2*t + 2) * (x + 4*t^11 + t^10 + 4*t^9 + 4*t^8 + 4*t^7 + 4*t^5 + 3*t^4 + t^3 + 3*t + 3) * (x + 2*t^11 + 3*t^10 + 2*t^9 + 3*t^8 + t^7 + t^6 + 2*t^5 + t^3 + 3*t^2 + 4*t + 3)
(2, 12, 2)
(x + 2) * (x + 3) * (x + 3*t^11 + 2*t^10 + 2*t^9 + 3*t^8 + 2*t^7 + t^6 + 3*t^5 + 2*t + 1) * (x + 4*t^11 + t^10 + 3*t^9 + t^7 

(42, 12, 2)
(2) * (x + 3*t^11 + 2*t^10 + 2*t^9 + 3*t^8 + 2*t^7 + t^6 + 3*t^5 + 2*t + 1) * (x + 4*t^11 + t^10 + 3*t^9 + t^7 + 2*t^5 + t^4 + 2*t^2 + 4*t + 1) * (x + t^11 + 2*t^10 + 2*t^9 + 3*t^8 + t^7 + 4*t^5 + 4*t^4 + 3*t^3 + t^2 + 2) * (x + 4*t^11 + 3*t^10 + 3*t^9 + 2*t^8 + 4*t^7 + t^5 + t^4 + 2*t^3 + 4*t^2 + 3) * (x + t^11 + 4*t^10 + 2*t^9 + 4*t^7 + 3*t^5 + 4*t^4 + 3*t^2 + t + 4) * (x + 2*t^11 + 3*t^10 + 3*t^9 + 2*t^8 + 3*t^7 + 4*t^6 + 2*t^5 + 3*t + 4) * (x + 1)^16 * (x + 4)^16 * (x + 2)^17 * (x + 3)^17
(43, 12, 2)
(4) * (x + 1)^18 * (x + 2)^18 * (x + 3)^18 * (x + 4)^18
(44, 12, 4)
(3) * (x + 2)^18 * (x + 3)^18 * (x + 1)^20 * (x + 4)^20
(45, 12, 3)
(3) * (x + t^11 + 4*t^10 + t^9 + 3*t^8 + 4*t^7 + 2*t^6 + t^5 + 4*t^4 + t^2 + 2*t) * (x + 4*t^11 + t^10 + 4*t^9 + 2*t^8 + t^7 + 3*t^6 + 4*t^5 + t^4 + 4*t^2 + 3*t) * (x + 2*t^11 + 3*t^10 + 2*t^9 + 4*t^8 + 2*t^6 + 2*t^5 + t^4 + 4*t^3 + t^2 + 4*t + 2) * (x + 3*t^11 + 2*t^10 + 3*t^9 + t^8 + 3*t^6 + 3*t^5 + 4*t^4 + t^3 + 4*t^2 + t + 3) * (x + 1)^

(x + 3*t^11 + 2*t^10 + 2*t^9 + 3*t^8 + 2*t^7 + t^6 + 3*t^5 + 2*t + 1) * (x + 4*t^11 + t^10 + 3*t^9 + t^7 + 2*t^5 + t^4 + 2*t^2 + 4*t + 1) * (x + t^11 + 2*t^10 + 2*t^9 + 3*t^8 + t^7 + 4*t^5 + 4*t^4 + 3*t^3 + t^2 + 2) * (x + 4*t^11 + 3*t^10 + 3*t^9 + 2*t^8 + 4*t^7 + t^5 + t^4 + 2*t^3 + 4*t^2 + 3) * (x + t^11 + 4*t^10 + 2*t^9 + 4*t^7 + 3*t^5 + 4*t^4 + 3*t^2 + t + 4) * (x + 2*t^11 + 3*t^10 + 3*t^9 + 2*t^8 + 3*t^7 + 4*t^6 + 2*t^5 + 3*t + 4) * (x + 1)^32 * (x + 4)^32 * (x + 2)^33 * (x + 3)^33
(83, 12, 1)
(3) * (x + 1)^34 * (x + 2)^34 * (x + 3)^34 * (x + 4)^34
(84, 12, 3)
(4) * (x + 2)^34 * (x + 3)^34 * (x + 1)^36 * (x + 4)^36
(85, 12, 4)
(2) * (x + t^11 + 4*t^10 + t^9 + 3*t^8 + 4*t^7 + 2*t^6 + t^5 + 4*t^4 + t^2 + 2*t) * (x + 4*t^11 + t^10 + 4*t^9 + 2*t^8 + t^7 + 3*t^6 + 4*t^5 + t^4 + 4*t^2 + 3*t) * (x + 2*t^11 + 3*t^10 + 2*t^9 + 4*t^8 + 2*t^6 + 2*t^5 + t^4 + 4*t^3 + t^2 + 4*t + 2) * (x + 3*t^11 + 2*t^10 + 3*t^9 + t^8 + 3*t^6 + 3*t^5 + 4*t^4 + t^3 + 4*t^2 + t + 3) * (x + 1)^34 * (x + 2)^34 * 

(115, 12, 1)
(4) * (x + t^11 + 4*t^10 + t^9 + 3*t^8 + 4*t^7 + 2*t^6 + t^5 + 4*t^4 + t^2 + 2*t) * (x + 4*t^11 + t^10 + 4*t^9 + 2*t^8 + t^7 + 3*t^6 + 4*t^5 + t^4 + 4*t^2 + 3*t) * (x + 2*t^11 + 3*t^10 + 2*t^9 + 4*t^8 + 2*t^6 + 2*t^5 + t^4 + 4*t^3 + t^2 + 4*t + 2) * (x + 3*t^11 + 2*t^10 + 3*t^9 + t^8 + 3*t^6 + 3*t^5 + 4*t^4 + t^3 + 4*t^2 + t + 3) * (x + 1)^46 * (x + 2)^46 * (x + 3)^46 * (x + 4)^46
(116, 12, 4)
(2) * (x + 3*t^11 + 2*t^10 + 3*t^9 + 2*t^8 + 4*t^7 + 4*t^6 + 3*t^5 + 4*t^3 + 2*t^2 + t + 2) * (x + t^11 + 4*t^10 + t^9 + t^8 + t^7 + t^5 + 2*t^4 + 4*t^3 + 2*t + 2) * (x + 4*t^11 + t^10 + 4*t^9 + 4*t^8 + 4*t^7 + 4*t^5 + 3*t^4 + t^3 + 3*t + 3) * (x + 2*t^11 + 3*t^10 + 2*t^9 + 3*t^8 + t^7 + t^6 + 2*t^5 + t^3 + 3*t^2 + 4*t + 3) * (x + 1)^46 * (x + 4)^46 * (x + 2)^47 * (x + 3)^47
(117, 12, 2)
(4) * (x + 3*t^11 + 2*t^10 + 2*t^9 + 3*t^8 + 2*t^7 + t^6 + 3*t^5 + 2*t + 1) * (x + 4*t^11 + t^10 + 3*t^9 + t^7 + 2*t^5 + t^4 + 2*t^2 + 4*t + 1) * (x + t^11 + 2*t^10 + 2*t^9 + 3*t^8 + t^7 + 4*t^5 + 4*

(3) * (x + t^11 + 4*t^10 + t^9 + 3*t^8 + 4*t^7 + 2*t^6 + t^5 + 4*t^4 + t^2 + 2*t) * (x + 4*t^11 + t^10 + 4*t^9 + 2*t^8 + t^7 + 3*t^6 + 4*t^5 + t^4 + 4*t^2 + 3*t) * (x + 2*t^11 + 3*t^10 + 2*t^9 + 4*t^8 + 2*t^6 + 2*t^5 + t^4 + 4*t^3 + t^2 + 4*t + 2) * (x + 3*t^11 + 2*t^10 + 3*t^9 + t^8 + 3*t^6 + 3*t^5 + 4*t^4 + t^3 + 4*t^2 + t + 3) * (x + 1)^58 * (x + 2)^58 * (x + 3)^58 * (x + 4)^58
(146, 12, 3)
(4) * (x + 3*t^11 + 2*t^10 + 3*t^9 + 2*t^8 + 4*t^7 + 4*t^6 + 3*t^5 + 4*t^3 + 2*t^2 + t + 2) * (x + t^11 + 4*t^10 + t^9 + t^8 + t^7 + t^5 + 2*t^4 + 4*t^3 + 2*t + 2) * (x + 4*t^11 + t^10 + 4*t^9 + 4*t^8 + 4*t^7 + 4*t^5 + 3*t^4 + t^3 + 3*t + 3) * (x + 2*t^11 + 3*t^10 + 2*t^9 + 3*t^8 + t^7 + t^6 + 2*t^5 + t^3 + 3*t^2 + 4*t + 3) * (x + 1)^58 * (x + 4)^58 * (x + 2)^59 * (x + 3)^59
(147, 12, 4)
(3) * (x + 3*t^11 + 2*t^10 + 2*t^9 + 3*t^8 + 2*t^7 + t^6 + 3*t^5 + 2*t + 1) * (x + 4*t^11 + t^10 + 3*t^9 + t^7 + 2*t^5 + t^4 + 2*t^2 + 4*t + 1) * (x + t^11 + 2*t^10 + 2*t^9 + 3*t^8 + t^7 + 4*t^5 + 4*t^4 + 3*t^3 +

(x + t^11 + 4*t^10 + t^9 + 3*t^8 + 4*t^7 + 2*t^6 + t^5 + 4*t^4 + t^2 + 2*t) * (x + 4*t^11 + t^10 + 4*t^9 + 2*t^8 + t^7 + 3*t^6 + 4*t^5 + t^4 + 4*t^2 + 3*t) * (x + 2*t^11 + 3*t^10 + 2*t^9 + 4*t^8 + 2*t^6 + 2*t^5 + t^4 + 4*t^3 + t^2 + 4*t + 2) * (x + 3*t^11 + 2*t^10 + 3*t^9 + t^8 + 3*t^6 + 3*t^5 + 4*t^4 + t^3 + 4*t^2 + t + 3) * (x + 1)^70 * (x + 2)^70 * (x + 3)^70 * (x + 4)^70
(176, 12, 1)
(4) * (x + 3*t^11 + 2*t^10 + 3*t^9 + 2*t^8 + 4*t^7 + 4*t^6 + 3*t^5 + 4*t^3 + 2*t^2 + t + 2) * (x + t^11 + 4*t^10 + t^9 + t^8 + t^7 + t^5 + 2*t^4 + 4*t^3 + 2*t + 2) * (x + 4*t^11 + t^10 + 4*t^9 + 4*t^8 + 4*t^7 + 4*t^5 + 3*t^4 + t^3 + 3*t + 3) * (x + 2*t^11 + 3*t^10 + 2*t^9 + 3*t^8 + t^7 + t^6 + 2*t^5 + t^3 + 3*t^2 + 4*t + 3) * (x + 1)^70 * (x + 4)^70 * (x + 2)^71 * (x + 3)^71
(177, 12, 4)
(4) * (x + 3*t^11 + 2*t^10 + 2*t^9 + 3*t^8 + 2*t^7 + t^6 + 3*t^5 + 2*t + 1) * (x + 4*t^11 + t^10 + 3*t^9 + t^7 + 2*t^5 + t^4 + 2*t^2 + 4*t + 1) * (x + t^11 + 2*t^10 + 2*t^9 + 3*t^8 + t^7 + 4*t^5 + 4*t^4 + 3*t^3 + t^2 +

In [10]:
import pickle
rfile = open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
bound=120
prime=5
residue=1
data=[]
for k in [1..len(s)-1]:
    index=s[k][0]
    if mod(index,prime)==residue:
        poly = s[k][1]
        poly=polynomialnumerator(D(poly))
        sd=leastSplittingDegree(poly,prime,bound)
        nt=numericalTermOverField(poly,prime,sd)
        print (index,sd,nt)
        data=data+[nt]
print data

(4) * (x + 2*t^3 + 4*t + 1) * (x + 2*t^3 + t^2 + t + 3) * (x + 3*t^3 + t + 4) * (x + 3*t^3 + 4*t^2 + 4*t + 2)
(1, 4, 4)
(3) * (x + 2*t^3 + 4*t + 1) * (x + 2*t^3 + t^2 + t + 3) * (x + 3*t^3 + t + 4) * (x + 3*t^3 + 4*t^2 + 4*t + 2) * (x + 1)^2 * (x + 2)^2 * (x + 3)^2 * (x + 4)^2
(6, 4, 3)
(x + 2*t^3 + 4*t + 1) * (x + 2*t^3 + t^2 + t + 3) * (x + 3*t^3 + t + 4) * (x + 3*t^3 + 4*t^2 + 4*t + 2) * (x + 1)^4 * (x + 2)^4 * (x + 3)^4 * (x + 4)^4
(11, 4, 1)
(x + 2*t^3 + 4*t + 1) * (x + 2*t^3 + t^2 + t + 3) * (x + 3*t^3 + t + 4) * (x + 3*t^3 + 4*t^2 + 4*t + 2) * (x + 1)^6 * (x + 2)^6 * (x + 3)^6 * (x + 4)^6
(16, 4, 1)
(3) * (x + 2*t^3 + 4*t + 1) * (x + 2*t^3 + t^2 + t + 3) * (x + 3*t^3 + t + 4) * (x + 3*t^3 + 4*t^2 + 4*t + 2) * (x + 1)^8 * (x + 2)^8 * (x + 3)^8 * (x + 4)^8
(21, 4, 3)
(3) * (x + 2*t^3 + 4*t + 1) * (x + 2*t^3 + t^2 + t + 3) * (x + 3*t^3 + t + 4) * (x + 3*t^3 + 4*t^2 + 4*t + 2) * (x + 1)^10 * (x + 2)^10 * (x + 3)^10 * (x + 4)^10
(26, 4, 3)
(x + 2*t^3 + 4*t + 1) * (x + 2*t^3 + t^2 + t

In [15]:
import pickle
rfile = open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
bound=120
prime=5
residue=1
data=[]
for k in [187..187]:
    index=s[k][0]
    if mod(index,prime)==residue:
        poly = s[k][1]
        poly=polynomialnumerator(D(poly))
        sd=leastSplittingDegree(poly,prime,bound)
        nt=numericalTermOverField(poly,prime,sd)
        print (index,sd,nt)
        data=data+[nt]
print data

(3) * (x + 2*t^3 + 4*t + 1) * (x + 2*t^3 + t^2 + t + 3) * (x + 3*t^3 + t + 4) * (x + 3*t^3 + 4*t^2 + 4*t + 2) * (x + 1)^74 * (x + 2)^74 * (x + 3)^74 * (x + 4)^74
(186, 4, 3)
[3]


In [14]:
import pickle
rfile = open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
bound=120
prime=5
for residue in [0..4]:
    data=[]
    for k in [1..len(s)-1]:
        index=s[k][0]
        if mod(index,prime)==residue:
            poly = s[k][1]
            poly=polynomialnumerator(D(poly))
            sd=leastSplittingDegree(poly,prime,bound)
            nt=numericalTermOverFieldNoPrint(poly,prime,sd)
            data=data+[nt]
    print data
    print

[3, 2, 1, 4, 1, 3, 1, 3, 4, 3, 3, 1, 3, 4, 3, 3, 1, 4, 3, 4, 2, 4, 4, 1, 4, 4, 1, 1, 4, 1, 4, 4, 3, 1, 1, 3, 4, 1, 2, 2]

[4, 3, 1, 1, 3, 3, 1, 1, 4, 3, 4, 3, 3, 2, 2, 3, 4, 2, 4, 1, 1, 4, 2, 4, 1, 1, 3, 3, 3, 3, 2, 2, 4, 4, 1, 1, 3, 3, 4, 4]

[2, 2, 2, 3, 1, 3, 3, 3, 2, 4, 2, 1, 2, 4, 1, 1, 1, 3, 1, 1, 2, 1, 1, 2, 4, 4, 1, 3, 1, 4, 1, 3, 3, 4, 4, 4, 3, 3, 1, 4]

[1, 2, 3, 1, 2, 1, 2, 3, 2, 4, 2, 4, 4, 3, 1, 3, 1, 2, 4, 3, 1, 1, 4, 4, 4, 4, 2, 1, 1, 3, 2, 1, 1, 1, 4, 4, 1, 1, 4, 1]

[1, 4, 2, 2, 1, 3, 2, 4, 4, 3, 2, 3, 3, 4, 1, 3, 3, 3, 1, 3, 4, 3, 3, 4, 2, 2, 1, 2, 1, 3, 2, 4, 2, 2, 1, 1, 4, 4, 2, 1]



In [41]:
import pickle
rfile = open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
bound=120

prime = 5
residue=0
for k in [1..len(s)-1]:
    index=s[k][0]
    if index!=0:
        if mod(index,prime)==residue:
            od=ord(prime,index-residue)
            poly = s[k][1]
            poly=polynomialnumerator(D(poly))
            sd=leastSplittingDegree(poly,prime,bound)
            nt=numericalTermOverFieldNoPrint(poly,prime,sd)
            print (index,sd,od,nt)


(5, 1, 1, 2)
(10, 1, 1, 1)
(15, 1, 1, 4)
(20, 1, 1, 1)
(25, 1, 2, 3)
(30, 1, 1, 1)
(35, 1, 1, 3)
(40, 1, 1, 4)
(45, 1, 1, 3)
(50, 1, 2, 3)
(55, 1, 1, 1)
(60, 1, 1, 3)
(65, 1, 1, 4)
(70, 1, 1, 3)
(75, 1, 2, 3)
(80, 1, 1, 1)
(85, 1, 1, 4)
(90, 1, 1, 3)
(95, 1, 1, 4)
(100, 1, 2, 2)
(105, 1, 1, 4)
(110, 1, 1, 4)
(115, 1, 1, 1)
(120, 1, 1, 4)
(125, 1, 3, 4)
(130, 1, 1, 1)
(135, 1, 1, 1)
(140, 1, 1, 4)
(145, 1, 1, 1)
(150, 1, 2, 4)
(155, 1, 1, 4)
(160, 1, 1, 3)
(165, 1, 1, 1)
(170, 1, 1, 1)
(175, 1, 2, 3)
(180, 1, 1, 4)
(185, 1, 1, 1)
(190, 1, 1, 2)
(195, 1, 1, 2)


In [42]:
import pickle
rfile = open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
bound=120
data1=[]
data2=[]
data3=[]
data4=[]
digits1=[]
digits2=[]
digits3=[]
digits4=[]
prime = 5
residue=0
for k in [1..len(s)-1]:
    index=s[k][0]
    dgt=index.digits(prime)
    if index!=0:
        if mod(index,prime)==residue:
            od=ord(prime,index-residue)
            poly = s[k][1]
            poly=polynomialnumerator(D(poly))
            sd=leastSplittingDegree(poly,prime,bound)
            nt=numericalTermOverFieldNoPrint(poly,prime,sd)
            if nt==1:
                data1=data1+[index.factor()]
                digits1=digits1+[dgt]
            if nt==2:
                data2=data2+[index.factor()]
                digits2=digits2+[dgt]
            if nt==3:
                data3=data3+[index.factor()]
                digits3=digits3+[dgt]
            if nt==4:
                data4=data4+[index.factor()]
                digits4=digits4+[dgt]
            

for k in [0..len(data1)-1]:
    print (1,data1[k])
for k in [0..len(data2)-1]:
    print (2,data2[k])
for k in [0..len(data3)-1]:
    print (3,data1[k])
for k in [0..len(data4)-1]:
    print (4,data4[k])


(1, 2 * 5)
(1, 2^2 * 5)
(1, 2 * 3 * 5)
(1, 5 * 11)
(1, 2^4 * 5)
(1, 5 * 23)
(1, 2 * 5 * 13)
(1, 3^3 * 5)
(1, 5 * 29)
(1, 3 * 5 * 11)
(1, 2 * 5 * 17)
(1, 5 * 37)
(2, 5)
(2, 2^2 * 5^2)
(2, 2 * 5 * 19)
(2, 3 * 5 * 13)
(3, 2 * 5)
(3, 2^2 * 5)
(3, 2 * 3 * 5)
(3, 5 * 11)
(3, 2^4 * 5)
(3, 5 * 23)
(3, 2 * 5 * 13)
(3, 3^3 * 5)
(3, 5 * 29)
(3, 3 * 5 * 11)
(4, 3 * 5)
(4, 2^3 * 5)
(4, 5 * 13)
(4, 5 * 17)
(4, 5 * 19)
(4, 3 * 5 * 7)
(4, 2 * 5 * 11)
(4, 2^3 * 3 * 5)
(4, 5^3)
(4, 2^2 * 5 * 7)
(4, 2 * 3 * 5^2)
(4, 5 * 31)
(4, 2^2 * 3^2 * 5)


In [43]:
import pickle
rfile = open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
bound=120
prime=5
residue=0
for k in [1..len(s)-1]:
    index=s[k][0]
    if index!=0:
        if mod(index,prime)==residue:
            od=ord(prime,index-residue)
            poly = s[k][1]
            poly=polynomialnumerator(D(poly))
            sd=leastSplittingDegree(poly,prime,bound)
            nt=numericalTermOverFieldNoPrint(poly,prime,sd)
            print ((index-residue)/5,od,nt)

(1, 1, 2)
(2, 1, 1)
(3, 1, 4)
(4, 1, 1)
(5, 2, 3)
(6, 1, 1)
(7, 1, 3)
(8, 1, 4)
(9, 1, 3)
(10, 2, 3)
(11, 1, 1)
(12, 1, 3)
(13, 1, 4)
(14, 1, 3)
(15, 2, 3)
(16, 1, 1)
(17, 1, 4)
(18, 1, 3)
(19, 1, 4)
(20, 2, 2)
(21, 1, 4)
(22, 1, 4)
(23, 1, 1)
(24, 1, 4)
(25, 3, 4)
(26, 1, 1)
(27, 1, 1)
(28, 1, 4)
(29, 1, 1)
(30, 2, 4)
(31, 1, 4)
(32, 1, 3)
(33, 1, 1)
(34, 1, 1)
(35, 2, 3)
(36, 1, 4)
(37, 1, 1)
(38, 1, 2)
(39, 1, 2)


In [44]:
import pickle
rfile = open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
bound=120
prime = 5
residue=0
for k in [1..len(s)-1]:
    index=s[k][0]
    if index!=0:
        if mod(index,prime)==residue:
            print
            od=ord(prime,index-residue)
            poly = s[k][1]
            poly=polynomialnumerator(D(poly))
            sd=leastSplittingDegree(poly,prime,bound)
            nt=numericalTermOverField(poly,prime,sd)
           


(2) * (x + 2)^2 * (x + 3)^2 * (x + 1)^4 * (x + 4)^4

(x + 2)^4 * (x + 3)^4 * (x + 1)^6 * (x + 4)^6

(4) * (x + 2)^6 * (x + 3)^6 * (x + 1)^8 * (x + 4)^8

(x + 2)^8 * (x + 3)^8 * (x + 1)^10 * (x + 4)^10

(3) * (x + 2)^10 * (x + 3)^10 * (x + 1)^12 * (x + 4)^12

(x + 2)^12 * (x + 3)^12 * (x + 1)^14 * (x + 4)^14

(3) * (x + 2)^14 * (x + 3)^14 * (x + 1)^16 * (x + 4)^16

(4) * (x + 2)^16 * (x + 3)^16 * (x + 1)^18 * (x + 4)^18

(3) * (x + 2)^18 * (x + 3)^18 * (x + 1)^20 * (x + 4)^20

(3) * (x + 2)^20 * (x + 3)^20 * (x + 1)^22 * (x + 4)^22

(x + 2)^22 * (x + 3)^22 * (x + 1)^24 * (x + 4)^24

(3) * (x + 2)^24 * (x + 3)^24 * (x + 1)^26 * (x + 4)^26

(4) * (x + 2)^26 * (x + 3)^26 * (x + 1)^28 * (x + 4)^28

(3) * (x + 2)^28 * (x + 3)^28 * (x + 1)^30 * (x + 4)^30

(3) * (x + 2)^30 * (x + 3)^30 * (x + 1)^32 * (x + 4)^32

(x + 2)^32 * (x + 3)^32 * (x + 1)^34 * (x + 4)^34

(4) * (x + 2)^34 * (x + 3)^34 * (x + 1)^36 * (x + 4)^36

(3) * (x + 2)^36 * (x + 3)^36 * (x + 1)^38 * (x + 4)^38

(4) * (x + 2)^38 

In [45]:
import pickle
rfile = open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
bound=120
prime = 5
residue=0
for k in [1..len(s)-1]:
    index=s[k][0]
    if index!=0:
        if mod(index,prime)==residue:
            od=ord(prime,index-residue)
            poly = s[k][1]
            poly=polynomialnumerator(D(poly))
            sd=leastSplittingDegree(poly,prime,bound)
            td=totalDegreeOverField(poly,prime,sd)
            nt=numericalTermOverFieldNoPrint(poly,prime,sd)
            print (index,td,nt)
           

(5, 12, 2)
(10, 20, 1)
(15, 28, 4)
(20, 36, 1)
(25, 44, 3)
(30, 52, 1)
(35, 60, 3)
(40, 68, 4)
(45, 76, 3)
(50, 84, 3)
(55, 92, 1)
(60, 100, 3)
(65, 108, 4)
(70, 116, 3)
(75, 124, 3)
(80, 132, 1)
(85, 140, 4)
(90, 148, 3)
(95, 156, 4)
(100, 164, 2)
(105, 172, 4)
(110, 180, 4)
(115, 188, 1)
(120, 196, 4)
(125, 204, 4)
(130, 212, 1)
(135, 220, 1)
(140, 228, 4)
(145, 236, 1)
(150, 244, 4)
(155, 252, 4)
(160, 260, 3)
(165, 268, 1)
(170, 276, 1)
(175, 284, 3)
(180, 292, 4)
(185, 300, 1)
(190, 308, 2)
(195, 316, 2)


In [46]:
import pickle
rfile = open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
bound=120
sd=12
prime = 5
residue=0
for k in [1..len(s)-1]:
    index=s[k][0]
    if index!=0:
        if mod(index,prime)==residue:
            od=ord(prime,index-residue)
            poly = s[k][1]
            poly=polynomialnumerator(D(poly))
            td=totalDegreeOverField(poly,prime,sd)
            nt=numericalTermOverFieldNoPrint(poly,prime,sd)
            print (index,td,nt)

(5, 12, 2)
(10, 20, 1)
(15, 28, 4)
(20, 36, 1)
(25, 44, 3)
(30, 52, 1)
(35, 60, 3)
(40, 68, 4)
(45, 76, 3)
(50, 84, 3)
(55, 92, 1)
(60, 100, 3)
(65, 108, 4)
(70, 116, 3)
(75, 124, 3)
(80, 132, 1)
(85, 140, 4)
(90, 148, 3)
(95, 156, 4)
(100, 164, 2)
(105, 172, 4)
(110, 180, 4)
(115, 188, 1)
(120, 196, 4)
(125, 204, 4)
(130, 212, 1)
(135, 220, 1)
(140, 228, 4)
(145, 236, 1)
(150, 244, 4)
(155, 252, 4)
(160, 260, 3)
(165, 268, 1)
(170, 276, 1)
(175, 284, 3)
(180, 292, 4)
(185, 300, 1)
(190, 308, 2)
(195, 316, 2)


In [49]:
import pickle
rfile = open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
bound=120
sd=12
prime = 5
residue=0
data=[]
for k in [1..len(s)-1]:
    index=s[k][0]
    if index-residue!=0:
        od=ord(prime,index-residue)
        poly = s[k][1]
        poly=polynomialnumerator(D(poly))
        td=totalDegreeOverField(poly,prime,sd)
        nt=numericalTermOverFieldNoPrint(poly,prime,sd)
        if td==nt:
            data=data+[index]
            print (index,nt)
print data

(1, 4)
(5, 2)
(12, 2)
(19, 2)
(20, 1)
(29, 3)
(39, 4)
(43, 2)
(51, 4)
(62, 2)
(63, 4)
(77, 1)
(78, 3)
(79, 3)
(83, 1)
(90, 3)
(96, 1)
(108, 1)
(113, 4)
(121, 1)
(125, 4)
(134, 1)
(141, 3)
(145, 1)
(150, 4)
(152, 1)
(156, 2)
(158, 1)
(169, 2)
(170, 1)
(171, 1)
(183, 1)
(189, 4)
(194, 2)
[1, 5, 12, 19, 20, 29, 39, 43, 51, 62, 63, 77, 78, 79, 83, 90, 96, 108, 113, 121, 125, 134, 141, 145, 150, 152, 156, 158, 169, 170, 171, 183, 189, 194]


In [51]:
import pickle
rfile = open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
bound=120
sd=12
prime = 5
residue=0
data=[]
for k in [1..len(s)-1]:
    index=s[k][0]
    if index-residue!=0:
        od=ord(prime,index-residue)
        poly = s[k][1]
        poly=polynomialnumerator(D(poly))
        td=totalDegreeOverField(poly,prime,sd)
        data=data+[td]
print data

[4, 6, 8, 8, 12, 12, 14, 16, 16, 20, 20, 22, 24, 24, 28, 28, 30, 32, 32, 36, 36, 38, 40, 40, 44, 44, 46, 48, 48, 52, 52, 54, 56, 56, 60, 60, 62, 64, 64, 68, 68, 70, 72, 72, 76, 76, 78, 80, 80, 84, 84, 86, 88, 88, 92, 92, 94, 96, 96, 100, 100, 102, 104, 104, 108, 108, 110, 112, 112, 116, 116, 118, 120, 120, 124, 124, 126, 128, 128, 132, 132, 134, 136, 136, 140, 140, 142, 144, 144, 148, 148, 150, 152, 152, 156, 156, 158, 160, 160, 164, 164, 166, 168, 168, 172, 172, 174, 176, 176, 180, 180, 182, 184, 184, 188, 188, 190, 192, 192, 196, 196, 198, 200, 200, 204, 204, 206, 208, 208, 212, 212, 214, 216, 216, 220, 220, 222, 224, 224, 228, 228, 230, 232, 232, 236, 236, 238, 240, 240, 244, 244, 246, 248, 248, 252, 252, 254, 256, 256, 260, 260, 262, 264, 264, 268, 268, 270, 272, 272, 276, 276, 278, 280, 280, 284, 284, 286, 288, 288, 292, 292, 294, 296, 296, 300, 300, 302, 304, 304, 308, 308, 310, 312, 312, 316, 316, 318, 320, 320]


In [53]:
import pickle
rfile = open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
bound=120
data=[]
prime = 5
residue=1
for k in [1..len(s)-1]:
    index=s[k][0]
    if index!=0:
        if mod(index,prime)==residue:
            od=ord(prime,index-residue)
            poly = s[k][1]
            poly=polynomialnumerator(D(poly))
            sd=leastSplittingDegree(poly,prime,bound)
            nt=numericalTermOverFieldNoPrint(poly,prime,sd)
            data=data+[nt]
            print (index,sd,od,nt)
print data

(1, 4, +Infinity, 4)
(6, 4, 1, 3)
(11, 4, 1, 1)
(16, 4, 1, 1)
(21, 4, 1, 3)
(26, 4, 2, 3)
(31, 4, 1, 1)
(36, 4, 1, 1)
(41, 4, 1, 4)
(46, 4, 1, 3)
(51, 4, 2, 4)
(56, 4, 1, 3)
(61, 4, 1, 3)
(66, 4, 1, 2)
(71, 4, 1, 2)
(76, 4, 2, 3)
(81, 4, 1, 4)
(86, 4, 1, 2)
(91, 4, 1, 4)
(96, 4, 1, 1)
(101, 4, 2, 1)
(106, 4, 1, 4)
(111, 4, 1, 2)
(116, 4, 1, 4)
(121, 4, 1, 1)
(126, 4, 3, 1)
(131, 4, 1, 3)
(136, 4, 1, 3)
(141, 4, 1, 3)
(146, 4, 1, 3)
(151, 4, 2, 2)
(156, 4, 1, 2)
(161, 4, 1, 4)
(166, 4, 1, 4)
(171, 4, 1, 1)
(176, 4, 2, 1)
(181, 4, 1, 3)
(186, 4, 1, 3)
(191, 4, 1, 4)
(196, 4, 1, 4)
[4, 3, 1, 1, 3, 3, 1, 1, 4, 3, 4, 3, 3, 2, 2, 3, 4, 2, 4, 1, 1, 4, 2, 4, 1, 1, 3, 3, 3, 3, 2, 2, 4, 4, 1, 1, 3, 3, 4, 4]
